In [1]:
# use Python 3.7

In [2]:
'''
Does the lattice matter for extinction conditions or should it be chosen to be consistent with spacegroup?
'''

import TensorScatteringClass as ten
import numpy as np

class ExtinctionConditions(ten.TensorScatteringClass):  
    '''
    Extend TensorScatteringClass to look at extinction conditions
    Start with normal spacegroup conditions      
    '''
    
    def isRat(self, x):        # true if number is close to a fraction with small denominator
        from fractions import Fraction 
        if np.isclose(x, float(Fraction(x).limit_denominator(99)), rtol=1e-9):
            return True
        else:
            return False
        
    def common_irrat(self, r1, r2):
        # return (True/False, Q)
        # True if r1 and r2 have a common irrational Q
        # Value of Q
        if self.isRat(r2 - r1) or self.isRat(r2 + r1):
            #return (True, r1)
            return True
        else:
            #return (False, 0)
            return False
        
    def find_rat(self, r, Q):
        # return (R, S)
        # R is the rational part of r (for irrational Q)
        # S is the sign of the irrational part
        if self.isRat(r + Q):
            S = -1
            R = r + Q
        else:
            S = +1
            R = r - Q
        #return (R, S)
        return (R - np.floor(R), S)  # return values in first cell
    
    def coordlist_to_symbols(self, coords):
        #convert a list of numerical vector coordinates to a list of string representations with symbols for irrational numbers
        # e.g. coords could be self.allR
        
        from fractions import Fraction

        coord_list = list(np.array(coords).flatten())
        npts = len(coord_list)
        rat_list = [None] * npts

        for i in range(npts):
            if rat_list[i] == None: #hasn't been done yet
                if self.isRat(coord_list[i]):
                    rat_list[i] = [coord_list[i], 1, 0] # register as a pure rational
                else:
                    rat_list[i] = [0, 1, coord_list[i]] # register as a pure irrational then look for matches
                    for j in range(i+1, npts):            # now look for others with matching irrat
                        #print(j)
                        if rat_list[j] == None:        # check that this element hasn't been done...
                            #print('one to check')
                            if self.common_irrat(coord_list[i], coord_list[j]):   #  a good pairing...
                                R, S = self.find_rat(coord_list[j], coord_list[i]) # find rational part and sign
                                #print('found rat: j= %i R=%f S=%i' % (j, R, S))
                                rat_list[j] = [R, S, coord_list[i]] # write out element
                                #print(rat_list)

        symbols = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

        all_rats, all_signs, all_irrats = [], [], []

        for rat in rat_list:

            if rat[1] == -1:
                all_signs += ['-']
            elif rat[0] == 0:
                all_signs += ['']        
            else:
                all_signs += ['+']
            all_irrats += [rat[2]]

            all_rats += [str(Fraction(rat[0]).limit_denominator(99)).replace('0','')] # limit denominator to 99 for rounding errors

        for i in range(len(all_irrats)):  
            if not isinstance(all_irrats[i], str):  #not a string
                idxs = [j for j in range(len(all_irrats)) if all_irrats[j]==all_irrats[i]] # indices of all occurences of value
                #set all values at idxs to symbols.pop(0)
                symbol = symbols.pop(0)
                for idx in idxs:
                    all_irrats[idx] = symbol

        sym_list = [] # list of all symboloc representations of positions
        for i in range(len(rat_list)):
            sym_list += [all_rats[i] + all_signs[i] + all_irrats[i]] 

        return [sym_list[i:i+3] for i in range(0,len(sym_list),3)] # list of lists - same shape as allR

In [6]:
t = ExtinctionConditions(spacegroup_number = 18, wyckoff_letter = 'a', lattice = [1.0, 1.0, 1.0, 90, 90, 90])
sglist = t.spacegroup_list_from_genpos_list(t.symxyz) 
print(sglist)

R_wyck = t.sitevec
R_gen = np.array([np.random.rand(), np.random.rand(), np.random.rand()])
hkl = np.array([2, 1, 0])
print('\nhkl: ', hkl)
print('Wyckoff position', R_wyck)
print(t.SF_symmetry(R_wyck, hkl, sglist)) # sets t.allR to Wyckoff position
print('\nGeneral position', R_gen)
print(t.SF_symmetry(R_gen, hkl, sglist))  # sets t.allR to Wyckoff position

def vlprint(l):
    for thing in l:
        print(thing)

vlprint(t.allR)
sym = t.coordlist_to_symbols(t.allR)
vlprint(sym)


Crystal properties

                    CIF file:  Spacegroup # 18
                     Lattice:  1.000 1.000 1.000 90.00 90.00 90.00
                   All sites:  
               Site selected:  2 a
                 Site vector:  0.000 0.000 0.426
       No. of spacegroup ops:  4
      No. of sym ops at site:  2
 No. of equiv. sites in cell:  2
   No. of pg ops for crystal:  4
[[array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]]), array([0., 0., 0.]), 1], [array([[ 1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0., -1.]]), array([ 0.5,  0.5, -0. ]), 1], [array([[-1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0., -1.]]), array([ 0.5,  0.5, -0. ]), 1], [array([[-1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0.,  1.]]), array([-0., -0.,  0.]), 1]]

hkl:  [2 1 0]
Wyckoff position [0.         0.         0.42623268]

                         hkl:  [2.0, 1.0, 0.0]
                Site allowed:  No
   Structure factor for site:  0.00+0.00i
          Spacegrou

In [5]:
# convert to symbols? - done - see new extension to tencorcalcs class (above)

# next: calculate strcture factor and look for zeros
# (h, 0, 0) zero for h = odd
# 1. solve with mathematica
# 2. solve with sympy
# 3. other options?


# create mathematic expression to solve (e.g. solve_phase_1.nb)
# mathematica found no solutions for the expression below - check if there should be and try another example




In [20]:
#Mathematic expression to solve

Fstring = ''
for site in sym:
    Fstring += 'Exp[2 Pi I (%s) h] + Exp[2 Pi I (%s) k] + Exp[2 Pi I (%s) l] + ' % (site[0], site[1], site[2])
Fstring = Fstring[0:-2] + '== 0'     
print(Fstring)


Exp[2 Pi I (a) h] + Exp[2 Pi I (b) k] + Exp[2 Pi I (c) l] + Exp[2 Pi I (1/2+a) h] + Exp[2 Pi I (1/2-b) k] + Exp[2 Pi I (-c) l] + Exp[2 Pi I (1/2-a) h] + Exp[2 Pi I (1/2+b) k] + Exp[2 Pi I (-c) l] + Exp[2 Pi I (-a) h] + Exp[2 Pi I (-b) k] + Exp[2 Pi I (c) l] == 0
